In [1]:
import os
from pathlib import Path
import sys
from pathlib import Path

def get_root_path():
    """Always use the same, absolute (relative to root) paths

    which makes moving the notebooks around easier.
    """
    return Path(os.getcwd()).parent

PROJECT_DIR = Path(get_root_path())
assert PROJECT_DIR.exists(), PROJECT_DIR

if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))

#display_pdf("Using project at:", PROJECT_DIR)

import gzip
import os
import pickle
from pathlib import Path
from typing import List, Dict
from rag.types import DocStore

# Load store: docs (text), meta, ids, and stored model name ----
store_path = PROJECT_DIR / "st3405_data/index/st4305_store.pkl.gz"
if not Path(store_path).exists():
    raise FileNotFoundError(f"Store file not found at {store_path}")

with gzip.open(store_path, "rb") as f:
    store = pickle.load(f)

docs: List[str] = store["docs"]
meta: List[dict] = store["meta"]
ids:  List[str] = store["ids"]

id_to_meta = {ids[i]: {**meta[i], "doc_id": ids[i]} for i in range(len(ids))}
doc_store: DocStore = {ids[i]: docs[i] for i in range(len(ids))}

In [2]:
from rag.ollama import OllamaClient
from rag.locale_ptbr import SYSTEM_PROMPT, GENERATION_PROMPT, REFORMULATE_PROMPT
client = OllamaClient(model="llama3.2:3b")

In [3]:

#prompt = GENERATION_PROMPT.format(context="[1] Enable1: Ativa a Saída1", query="Qual o comando para habilitar a Saída 1")
#display(client.chat(prompt, temperature=0.0, system_prompt=SYSTEM_PROMPT))


In [4]:
# prompt = """
# <CONTEXT>\n[1] 20. ENVIO DE COMANDOS   20. 1. Lista de Comandos disponíveis  Preset: Apresenta as configurações atuais do equipamento. PresetP: Apresenta as configurações do parâmetro solicitado. ReqConMntSvr: Solicita que o dispositivo se reporte ao servidor de manutenção para verificar se há atualizações. ReqIMSI: Solicita o IMSI (International Mobile Subscriber Identity). ReqICCID: Solicita o ICCID do SimCard. ReqConNtw: Solicita em qual rede o dispositivo está conectado (2G, 4G). ReqOwnNo: Solicita o número da linha SimCard inserido, caso esteja cadastrado. SetGoogleMap: Configura o formato do link Google Maps. ReqGoogleMap: Solicita o formato do link Google Maps. StatusReq: Solicita a posição do dispositivo. Reset: Reinicia o dispositivo. Reboot: Reinicia o dispositivo. ReqVer: Solicita a versão do FW, utilizar o valor 1 no campo: Option.\nEnable1: Ativa a Saída1 Disable1: Desativa a Saída1 Enable2: Ativa a Saída2 Disable2: Desativa a Saída2 Enable3: Ativa a Saída3 Disable3: Desativa a Saída3 EraseAll: Apaga as posições que foram armazenadas na memória do dispositivo e desabilita a saída (caso esteja ativada) do equipamento. SetOdometer: Configura o odometro do dispositivo. InitMsgNo: Reinicia o contador das mensagens. SetHMeter: Configura o horímetro do dispositivo. InitCircleGeo: Inicia / apaga todas as cercas circulares no dispositivo. InitAllPolygonGeo: Inicia / apaga todas as cercas poligonais no dispositivo. Request Server Lock: Solicita o status da função Server Lock. ReqSttAssignmap: Solicita a configuração do Mapeamento das STT. ReqAltAssignmap: Solicita a configuração do Mapeamento das ALT. Encoding Type: Este comando configura o Encoding Type. O ID do dispositivo pode ser encriptado quando um comando é enviado por SMS. Favor observar a tabela abaixo: Valor Tipo de Controle de Saída 0 Não ativa nenhuma saída. 1 Ativa ambas Saídas 1 e Saída 2. 2 Ativa somente Saída 2. 3 Ativa somente Saída 1. GetDidOutCtrl: Este comando é para requisitar o tipo de controle de saída em relação ao ID do Motorista (iButton) configurado previamente. Set GeoFence Area Jamming: Ativa a função de detecção de Jammer dentro de uma área de Cerca configurada. 0 para “Desativar” e 1 para “Ativar”. Get GeoFence Area Jamming: Consulta se a função de detecção de Jammer dentro de uma área de Cerca está ativa ou não. 0 desativada, 1 ativada. Set Buzzer pulse off: Caso o Buzzer seja habilitado devido a uma situação de Velocidade Alta (Over Speed configurado), o Buzzer não irá pulsar caso essa opção esteja habilitada. 1 para “Habilitar” e 0 para “Desabilitar”. Get Buzzer pulse off: Consulta se a função “Buzzer pulse off” está habilitada ou não no equipamento.\nGet anti theft status: Consulta o status do serviço de antifurto. 0 desativado, 1 ativado. ReqAllDriver: Comando para solicitar todos os IDs de Motorista configurados no equipamento. ReqPolyinfo: Solicita as informações das cercas poligonais configuradas. No campo “Opção” inserir o ID da cerca a ser requisitado. Set immobilizer Always Pulsed: Configura o comportamento do pulso do imobilizador. 0 desativado, 1 ativado. Caso ativado, o imobilizar vai pulsar independente do status de ignição caso haja uma emergência por antifurto. Set Immob. Cycle time: Configura o tempo do ciclo (ativação) do imobilizador. Valores 0~7200 em segundos. Req Immob. Cycle time: Solicita o tempo do ciclo do imobilizador configurado previamente. Set immob speed limit: Quando o Imobilizador é ativado com o veículo em movimento, o Imobilizador irá ativar quando a velocidade do veículo estiver menor do que a velocidade configurada neste comando. Possíveis valores: 0 a 300 Km / h. InitIDPolygonGeo: Inicia / apaga a cerca poligonal especifica no dispositivo. É possível apagar as cercas poligonais dentro de um range de IDs específico. Exemplo: no campo “Opção” insira, “1,20”. Dessa forma, todas as cercas entre o ID 1 e 20 serão excluídas do equipamento. InitIDCircularGeo: Inicia / apaga a cerca circular especifica no dispositivo. É possível apagar as cercas circulares dentro de um range de IDs específico. Exemplo: no campo “Opção” insira, “1,500”. Dessa forma, todas as cercas entre o ID 1 e 500 serão excluídas do equipamento. ReqCircInfo: Solicita as informações das cercas circulares configuradas. No campo “Opção” inserir o ID da cerca a ser requisitado. ActivateAntiTheft: Ativa a função antifurto. Req circular ID: Solicita quais IDs estão configurados com cerca circular no equipamento, por grupo de IDs. Exemplo: No campo “Opção”, insira o valor “1”. Dessa forma, o equipamento reporta todos os IDs configurados no grupo 1 (IDs de 1 a 100). Grupo 1: Cerca eletrônica circular de ID 1 a 100 Grupo 2: Cerca eletrônica circular de ID 101 a 200 Grupo 3: Cerca eletrônica circular de ID 201 a 300 Grupo 10: Cerca eletrônica circular de ID 901~999. Start DPA Calibration: Inicia o modo de calibração da função DPA. Stop DPA Calibration: Finaliza o modo de calibração da função DPA.\nReqDPAParam: Solicita os parâmetros de DPA calibrados. ReqDPADefault: Solicita os parâmetros padrões do DPA. InitParkOdometer: Inicializa o odometro do dispositivo. InitAllDriverID: Este comando é utilizado para apagar todos os IDs de Motorista (iButton) configurados no equipamento. SetVol: Este comando é utilizado para configurar o volume do speaker. Valores validos: 0 (volume mínimo) a 6 (volume máximo). ReqVol: Este comando solicita o valor do volume do speaker. Valores validos: 0 (volume mínimo) a 6 (volume máximo). SetMicVol: Este comando é utilizado para configurar o volume do microfone. Valores validos: 0 (volume mínimo) a 15 (volume máximo). ReqMicVol: Este comando solicita o valor do volume do microfone. Valores validos: 0 (volume mínimo) a 6 (volume máximo).\n\n[2] 32. Calibração DPA por Comando  Para que a calibração via comando seja realizada com sucesso, no parametro Acelerometro ative a função DPA(1910) e siga os passos a seguir. Envie o comando: Start DPA Calibration. Os Leds GPS e GPRS piscam por 0. 5s continuamente. Dirija o veiculo em uma velocidade acima de 10 km / h, e realize 3 freadas, 3 acelarações e 3 curvas em ritmos considerados normais. Envie o comando: Stop DPA Calibration. Os Leds GPS e GPRS piscam como fixo normalmente. O rastreador enviará o evento de calibração concluída.\n</CONTEXT>\n\n<QUESTION>\nQual o comando para reiniciar o dispositivo via envio de comandos?\n</QUESTION>\n\n<GUIDELINES>\n1) Fonte única:\n   - Use EXCLUSIVAMENTE o conteúdo de <CONTEXT>.\n   - Se faltar informação essencial, responda exatamente:\n     <BEGIN_ANSWER>\n     Não encontrado no contexto.\n     </END_ANSWER>\n\n2) Estruturas possíveis (seção técnica genérica):\n   a) Comandos ou instruções diretas: linhas de ação que começam por CMD;, PRG;, RES;, REQ; ou nomes simples (Enable1, Reset, Reboot, etc.).\n   b) Parâmetros e configurações: nome + ID (se houver) + descrição + valores possíveis e efeitos técnicos.\n   c) Procedimentos ou operações: passo a passo técnico (1., 2., 3., …) com condições e resultados.\n   d) Ligações elétricas: fios, pinos, conectores e seus sinais (VCC, GND, TX, RX…).\n   e) Diagnóstico e status: tabelas de LEDs, mensagens de erro, indicadores de módulo, códigos, sensores, modos de operação.\n   f) Alertas e eventos: condições que disparam notificações, ID de alerta, ações associadas.\n   g) Calibração ou inicialização: sequências de comandos ou ações físicas para ajuste de sensores.\n\n3) Regras de extração e prioridade:\n   a) Se existir comando literal que resolve a pergunta, liste-o exatamente como aparece.\n   b) Se não houver comando direto, mas houver parâmetros equivalentes (nome, ID, valor), extraia apenas os necessários.\n   c) Se for um procedimento ou sequência operacional, resuma em passos numerados.\n   d) Se for ligação ou diagnóstico, apresente pares (função ↔ pino/sinal) ou tabela simples de status.\n   e) Se múltiplos itens forem relevantes, apresente todos agrupados por tipo.\n\n4) Regra anti-paráfrase e fidelidade:\n   - Copie linhas estruturadas literalmente.\n   - Não traduza nem altere a forma de comandos ou IDs.\n   - Não invente campos, unidades, nem páginas.\n   - Não crie prefixos como “CMD;” quando o comando for simples (ex.: Enable1).\n   - Não adicione colchetes ou símbolos extras.\n   - Não descreva o contexto — apenas extraia e formate.\n\n5) Formato obrigatório da resposta:\n<BEGIN_ANSWER>\n[resumo técnico breve, objetivo e impessoal, sem repetir o contexto]\n[se houver comandos, use este bloco EXATO:]\n```txt\n<linha literal 1>\n<linha literal 2>   # se houver múltiplas\n```\n[se houver parâmetros ou tabelas, apresente com colunas: Nome | ID | Valor | Efeito técnico]\n[se houver sequência, numere os passos de forma concisa]\n[sempre que citar um dado específico (valor, parâmetro, comando, ID), inclua a fonte entre colchetes [n] de acordo com as seções do CONTEXTO]\n</END_ANSWER>\n\n⚠️ Importante:\n- Sempre inicie a resposta com <BEGIN_ANSWER> e finalize com </END_ANSWER>\n- A resposta deve obrigatoriamente conter AMBAS as tags, mesmo que o conteúdo seja curto.\n\n7) Proibições absolutas:\n- Não escreva fora de <BEGIN_ANSWER>…</END_ANSWER> (nem antes nem depois).\n- Não explique as instruções nem resuma o contexto.\n- Não cite marca, modelo ou fabricante, exceto se constarem no CONTEXTO.\n- Não adicione interpretações, exemplos inventados, nem traduções.\n- Não modifique o formato de unidades, IDs, comandos ou rótulos originais.\n\n8) Critério de clareza mínima:\n   Se o CONTEXTO contiver múltiplos tópicos, selecione apenas o(s) que respondem à pergunta,\n   priorizando precisão e concisão técnica sobre volume de texto.\n\n</GUIDELINES> 
# """
prompt=REFORMULATE_PROMPT.format(query="Como consultar a versão do firmware do rastreador por comando?",vc=5,vl=5,te=10)


resposta = client.generate2(prompt,temperature=0.5)
print(resposta)


 Você é um assistente de modelo de linguagem de IA especializado em Rastreadores de veiculos.
 Sua tarefa é gerar questoes diferentes da pergunta do usuário em português do Brasil para recuperar documentos relevantes de um banco de dados vetorial. Ao gerar múltiplas perspectivas sobre a pergunta do usuário, seu objetivo é ajudar o usuário a superar algumas das limitações da busca por similaridade baseada em distância. Forneça estas perguntas alternativas separadas por novas linhas.
- Produza JSON com as chaves: O JSON deve ter **EXATAMENTE** as três chaves a seguir:
  {
    "variantes_curtas": [ ... ],        // 5 itens; Ate 6 palavras; foco em termos-chave
    "variantes_longas": [ ... ],        // 5 itens; 6–16 palavras; frases técnicas
    "termos_exatos": [ ... ],            // 10 itens; Comandos, parâmetros
    "siglas": [ ... ] // **10** siglas; ate 5 letras
  }
**Instruções de Formato OBRIGATÓRIO (JSON):**
- Mantenha o mesmo objetivo sem derivar o tema (evitar drift).
- Inclua 

In [5]:
# resposta2 = client.generate2("cite as fontes")
# resposta2